In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Getting top 50 companies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
# Define the User-Agent string
headers = {
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Mobile Safari/537.36'
}

# Make a GET request to a URL with the custom User-Agent
url = 'https://finance.yahoo.com/quote/%5ENSEI/components/'
response = requests.get(url, headers=headers)


table_data = []
col_name = []
count=0
soup = BeautifulSoup(response.content, 'html.parser')
for i in soup.find_all("table"):
    rows = i.find_all('tr')  # Find all rows in the table
    for row in rows:
        cols = row.find_all('td')  # Find all columns (td elements)
        cols = [col.text.strip() for col in cols]  # Extract text and remove extra spaces
        table_data.append(cols)

table_data = table_data[1:]
items = [item[0] for item in table_data]

In [ ]:
# Getting 10 years open and closed price for each companies
list_df = []
sum = 0
file_names = []  # Names for CSV files inside the zip
for item in items:
  url = f'https://finance.yahoo.com/quote/{item}/history/?frequency=1d&period1=1388534400&period2=1725779612'
  response = requests.get(url, headers=headers)

  table_data = []
  col_name = []

  soup = BeautifulSoup(response.content, 'html.parser')
  for i in soup.find_all("table"):
      rows = i.find_all('tr')  # Find all rows in the table
      for row in rows:
          cols = row.find_all('th')  # Find all columns (th elements)
          cols = [col.text.strip() for col in cols]  # Extract text and remove extra spaces
          col_name.append(cols)
          cols = row.find_all('td')  # Find all columns (td elements)
          cols = [col.text.strip() for col in cols]  # Extract text and remove extra spaces
          table_data.append(cols)

  col_name = col_name[0]
  col_name = [col.split()[0] for col in col_name]

  df = pd.DataFrame(data=table_data, columns=col_name)
  df = df.dropna().reset_index(drop = True)
  list_df.append(df)

  # csv_buffer = io.StringIO()
  # df.to_csv(csv_buffer, index=False)

  # # Get the size of the CSV content (in bytes)
  # csv_size = len(csv_buffer.getvalue().encode('utf-8'))
  # # print(csv_size) # size in bytes
  # sum = sum + csv_size
  item = item.split('.')[0]
  file_names.append(item)
  # df.to_csv(f'/content/drive/My Drive/AbhiData/{item}.csv', index=False)
  # print(table_data)

In [ ]:
print(sum)

5261314


In [ ]:
# Saving all the csv files into a zip file
import io
import zipfile

# Create a ZIP file in-memory
zip_buffer = io.BytesIO()

with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zf:
    for df, name in zip(list_df, file_names):
        # Write each dataframe to a CSV format in-memory
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)

        # Add the CSV data to the ZIP archive
        zf.writestr(name, csv_buffer.getvalue())

# Save the ZIP to a file
with open('/content/drive/My Drive/AbhiData/TopFiftyCompanies.zip', 'wb') as f:
    f.write(zip_buffer.getvalue())

print("ZIP file with dataframes created successfully.")

ZIP file with dataframes created successfully.


In [ ]:
zip_file_path = '/content/drive/My Drive/AbhiData/TopFiftyCompanies.zip'

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zf:
    # List files in the ZIP archive
    print(zf.namelist())  # This will list the files inside the ZIP

    # Specify the file you want to read (e.g., 'file1.csv')
    with zf.open('RELIANCE') as file:
        # Read the file into a pandas DataFrame
        df = pd.read_csv(file)
# print(df)

['CIPLA', 'BHARTIARTL', 'RELIANCE', 'LTIM', 'APOLLOHOSP', 'TCS', 'MARUTI', 'WIPRO', 'HEROMOTOCO', 'ONGC', 'BAJAJ-AUTO', 'INDUSINDBK', 'TATASTEEL', 'ITC', 'ADANIENT', 'HDFCLIFE', 'KOTAKBANK', 'COALINDIA', 'ULTRACEMCO', 'TATACONSUM', 'TITAN', 'NESTLEIND', 'BRITANNIA', 'LT', 'SHRIRAMFIN', 'HINDALCO', 'BAJAJFINSV', 'NTPC', 'BAJFINANCE']
